In [15]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

What's happening:

df.itertuples() converts each row into a named tuple where you can access values like:

row.name for city name
row.lat for latitude
row.lon for longitude


combinations(..., 2) takes these tuples and creates all possible pairs of cities. The 2 means "make pairs of 2"
In the loop:

c1 gets the first city in the pair
c2 gets the second city in the pair

In [38]:
data = {
    'name': ['Rome', 'Milan', 'Naples', 'Turin', 'Palermo', 'Genoa'],
    'lat': [41.9028, 45.4642, 40.8518, 45.0703, 38.1157, 44.4056],
    'lon': [12.4964, 9.1900, 14.2681, 7.6869, 13.3619, 8.9463]
}

df = pd.DataFrame(data)
# using itertools to generate all possible combinations of 2 cities
for row in df.itertuples():
    print(f'Name is this {row.name}')
# With itertools instead of saying let's say df['name'] we can say row.name. 
print(df.head())
DIST_MATRIX = np.zeros((len(df), len(df)))
# See what combinations does
for city1, city2 in combinations(df.itertuples(), 2):
    DIST_MATRIX[city1.Index, city2.Index] = DIST_MATRIX[city2.Index, city1.Index] = geodesic((city1.lat, city1.lon), (city2.lat, city2.lon)).km

print(DIST_MATRIX)


Name is this Rome
Name is this Milan
Name is this Naples
Name is this Turin
Name is this Palermo
Name is this Genoa
      name      lat      lon
0     Rome  41.9028  12.4964
1    Milan  45.4642   9.1900
2   Naples  40.8518  14.2681
3    Turin  45.0703   7.6869
4  Palermo  38.1157  13.3619
[[  0.         477.02546129 188.6475709  524.43297713 426.93694824
  400.78665276]
 [477.02546129   0.         657.84687879 125.82100593 886.47772698
  119.20497669]
 [188.6475709  657.84687879   0.         712.28589073 313.61428581
  588.33906504]
 [524.43297713 125.82100593 712.28589073   0.         905.2636729
  124.12030586]
 [426.93694824 886.47772698 313.61428581 905.2636729    0.
  790.20063127]
 [400.78665276 119.20497669 588.33906504 124.12030586 790.20063127
    0.        ]]


## Geodesic Distance Calculation

The geodesic distance between two points is the same regardless of the terrain:

- Water (e.g., Mediterranean Sea)
- Mountains (e.g., Alps)
- Flat land

```python
distance = geodesic(milan, palermo).km  # ≈ 886.48 km

## Why Use the Geodesic Formula?

The geodesic formula takes into account several important factors:

- Earth's radius
- The curvature of the Earth
- The fact that Earth is slightly squished (ellipsoid)
- The changing distance between longitude lines

That's why if you were planning a long flight route, it might look curved on a flat map, but it's actually the shortest path on our spherical Earth!

# On a flat map (like Google Maps), it looks like you should go in a straight line
# But because Earth is round (like the orange), the actual shortest path curves slightly!

# That's why:
```python
flat_distance = flat_earth_distance(rome, milan)    # Wrong way (as if Earth was flat)
curved_distance = geodesic(rome, milan).km          # Right way (considers Earth's curve)

In [33]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))

for c1, c2 in combinations(CITIES.itertuples(), 2):
    # This is doing two assignments at once because the distance from city A to city B is the same as the distance from city B to city A. Let's break it into steps:
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()
print(DIST_MATRIX)



[[  0.         349.30401144 391.04186819 ... 223.61470849 285.67481712
  266.79616501]
 [349.30401144   0.          50.17877215 ... 566.29615596 634.93310475
  614.95900451]
 [391.04186819  50.17877215   0.         ... 604.01577185 676.49916849
  654.72206469]
 ...
 [223.61470849 566.29615596 604.01577185 ...   0.         104.8569216
   63.17924401]
 [285.67481712 634.93310475 676.49916849 ... 104.8569216    0.
   44.69517255]
 [266.79616501 614.95900451 654.72206469 ...  63.17924401  44.69517255
    0.        ]]


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

## Fast Greedy Algorithm 


In [57]:
current_city = 0  # Starting city (you can choose any starting point)
visited = [False] * len(DIST_MATRIX)  # Initialize visited list
visited[current_city] = True  # Mark the starting city as visited
total_distance = 0  # To accumulate total distance

# Loop until all cities are visited
while not all(visited):
    min_value = np.inf
    min_index = -1

    for j in range(len(DIST_MATRIX[current_city])):  # Loop through cities from the current city
        if DIST_MATRIX[current_city, j] < min_value and not visited[j] and DIST_MATRIX[current_city, j] != 0:
            min_value = DIST_MATRIX[current_city, j]
            min_index = j

    if min_index != -1:  # Check if a minimum index was found
        visited[min_index] = True  # Mark the new city as visited
        total_distance += min_value  # Add the distance to total
        print(f'From city {current_city} to city {min_index} is {min_value} km')
        current_city = min_index  # Update the current city to the newly found city
        
    if all(visited):
        total_distance += DIST_MATRIX[current_city, 0]
        print(f'From city {current_city} to city 0 is {DIST_MATRIX[current_city, 0]} km')
        print(f'Total distance: {total_distance} km')


From city 0 to city 2 is 188.64757090139386 km
From city 2 to city 4 is 313.61428581460615 km
From city 4 to city 5 is 790.2006312734908 km
From city 5 to city 1 is 119.20497669118207 km
From city 1 to city 3 is 125.82100593033576 km
From city 3 to city 0 is 524.4329771341531 km
Total distance: 2061.921447745162 km
